In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# task 1
energy = pd.read_excel("En_In.xls", skiprows=17)
pd.set_option("future.no_silent_downcasting", True)
energy.drop(energy.columns[[0, 1]], axis=1, inplace=True)

energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]

/usr/local/lib/python3.11/dist-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/usr/local/lib/python3.11/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [4]:
# task 2
energy["Energy Supply"] = energy["Energy Supply"].replace(r"\.", np.nan, regex=True)
energy["Energy Supply"] *= 1000000

energy

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN


In [ ]:
# task 3
to_rename = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong",
}
energy["Country"] = energy["Country"].replace(to_rename)

energy

In [ ]:
# task 4
energy["Country"] = (
    energy["Country"]
    .str.replace(r"\(.*\)|\d+", "", regex=True)
    .replace(r"\s+$", "", regex=True)
)

energy.loc[energy["Country"].isin(["American Samoa", "South Korea", "Bolivia"])]


In [ ]:
# task 5
gpd = pd.read_csv("gpd.csv", skiprows=4)

to_rename = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong",
}

gpd["Country Name"] = gpd["Country Name"].replace(to_rename)

gpd.head(1)

In [ ]:
# task 6
scimagojr = pd.read_excel("scimagojr.xlsx")


In [ ]:
# task 7
scimagojr_top15 = scimagojr[scimagojr["Rank"] <= 15]

gpd.rename(columns={"Country Name": "Country"}, inplace=True)

gpd_2006_2015 = gpd[["Country"] + [str(year) for year in range(2006, 2016)]]
gpd_2006_2015_top15 = gpd_2006_2015[
    gpd_2006_2015["Country"].isin(scimagojr_top15["Country"])
]

energy_top15 = energy[energy["Country"].isin(scimagojr_top15["Country"])]


scimagojr_energy_merged = pd.merge(scimagojr, energy_top15, on="Country")
final_merged = pd.merge(scimagojr_energy_merged, gpd_2006_2015_top15, on="Country")
final_merged.set_index("Country", inplace=True)

print(final_merged.head(3))
print(final_merged.shape)
print(final_merged.columns)

In [ ]:
# task 8
def avg_gpd():
    return (
        final_merged[[str(year) for year in range(2006, 2016)]]
        .mean(axis=1)
        .sort_values(ascending=False)
    )


avg_gpd()

In [ ]:
# task 9
def gpd_delta():
    fifth_country = avg_gpd().index[4]
    gdp_2006 = final_merged.loc[fifth_country, "2006"]
    gdp_2015 = final_merged.loc[fifth_country, "2015"]
    gdp_change = gdp_2015 - gdp_2006
    return (fifth_country, gdp_change)


gpd_delta()

In [ ]:
# task 10
def max_renewable_percent():
    max_renewable_country = final_merged["% Renewable"].idxmax()
    return (
        max_renewable_country,
        final_merged.loc[max_renewable_country, "% Renewable"],
    )


max_renewable_percent()

In [ ]:
# task 11
def add_population():
    final_merged["Estimated population"] = final_merged["Energy Supply"] / final_merged[
        "Energy Supply per Capita"
    ].replace(0, np.nan)


def get_six_country_by_population():
    population_sorted = final_merged["Estimated population"].sort_values(
        ascending=False
    )
    sixth_country = population_sorted.index[5]
    sixth_population = population_sorted.iloc[5]
    return (sixth_country, sixth_population)


add_population()
get_six_country_by_population()

In [ ]:
# task 12
def add_citations_per_person():
    final_merged["Citations per Person"] = (
        final_merged["Citations"] / final_merged["Estimated population"]
    )


def get_correlation_task_12():
    correlation = final_merged["Citations per Person"].corr(
        final_merged["Energy Supply per Capita"]
    )
    return correlation


add_citations_per_person()
get_correlation_task_12()

In [ ]:
# task 13
def add_renewable_above_median():
    median = final_merged["% Renewable"].median()
    final_merged["Renewable Above Median"] = (
        final_merged["% Renewable"] >= median
    ).astype(int)


def get_renewable_above_median_to_rank():
    return final_merged[["Renewable Above Median", "Rank"]].sort_values(by="Rank")


add_renewable_above_median()
get_renewable_above_median_to_rank()

In [ ]:
# task 14
ContinentDict = {
    "China": "Asia",
    "United States": "North America",
    "Japan": "Asia",
    "United Kingdom": "Europe",
    "Russian Federation": "Europe",
    "Canada": "North America",
    "Germany": "Europe",
    "India": "Asia",
    "France": "Europe",
    "South Korea": "Asia",
    "Italy": "Europe",
    "Spain": "Europe",
    "Iran": "Asia",
    "Australia": "Australia",
    "Brazil": "South America",
}


def add_continent():
    final_merged["Continent"] = final_merged.index.to_series().map(ContinentDict)


def groupby_continent():
    population_stats = final_merged.groupby("Continent")["Estimated population"].agg(
        ["size", "sum", "mean", "std"]
    )
    return population_stats


add_continent()
groupby_continent()

In [ ]:
# task 15
continent_colors = {
    "Asia": "red",
    "Europe": "green",
    "North America": "blue",
    "South America": "yellow",
    "Australia": "purple",
}

def show_chart():
    final_merged["Color"] = final_merged["Continent"].map(continent_colors)
    plt.figure(figsize=(10, 6))
    plt.scatter(
        final_merged["Rank"],
        final_merged["% Renewable"],
        s=final_merged["2015"] / 1000000000,  # Розмір бульбашки - ВВП за 2015 рік
        c=final_merged["Color"],
        alpha=0.6,
        edgecolor="w",
        linewidth=0.5,
    )
    plt.grid(True)
    plt.xlabel("Rank")
    plt.ylabel("% Renewable")
    plt.title("% Renewable vs. Rank (size by 2015 GDP, color by continent)")
    plt.show()


show_chart()